In [1]:
import torch
from torch import nn

# Configuration

In [5]:
import os
from os.path import join
import torch
from enum import Enum
# from parse import parse_args
import multiprocessing

# os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
# args = parse_args()

# ROOT_PATH = os.path.dirname(os.path.dirname(__file__))
# CODE_PATH = join(ROOT_PATH, 'code')
# DATA_PATH = join(ROOT_PATH, 'data')
# BOARD_PATH = join(CODE_PATH, 'runs')
# FILE_PATH = join(CODE_PATH, 'checkpoints')
# import sys
# sys.path.append(join(CODE_PATH, 'sources'))


# if not os.path.exists(FILE_PATH):
#     os.makedirs(FILE_PATH, exist_ok=True)

config = {}
all_dataset = ['lastfm', 'gowalla', 'yelp2018', 'amazon-book']
all_models  = ['mf', 'lgn']
# config['batch_size'] = 4096
config['bpr_batch_size'] = 2048 # the batch size for bpr loss training procedure"
config['latent_dim_rec'] = 64 # the embedding size of lightGCN
config['lightGCN_n_layers']= 3 # the layer num of lightGCN
config['dropout'] = 0 # using the dropout or not
config['keep_prob']  = 0.6 # the batch size for bpr loss training procedure
config['A_n_fold'] = 100 # the fold num used to split large adj matrix, like gowalla
config['test_u_batch_size'] = 100 # the batch size of users for testing
config['multicore'] = 0 # whether we use multiprocessing or not in test
config['lr'] = 0.001 # the learning rate
config['decay'] = 1e-4 # the weight decay for l2 normalizaton
config['pretrain'] = 0 # whether we use pretrained weight or not
config['A_split'] = False #
config['bigdata'] = False #

GPU = torch.cuda.is_available()
device = torch.device('cuda' if GPU else "cpu")
CORES = multiprocessing.cpu_count() // 2
seed = 2020

dataset = 'gowalla'
model_name = 'lgn'
if dataset not in all_dataset:
    raise NotImplementedError(f"Haven't supported {dataset} yet!, try {all_dataset}")
if model_name not in all_models:
    raise NotImplementedError(f"Haven't supported {model_name} yet!, try {all_models}")




TRAIN_epochs = 1000
LOAD = 0
PATH = "./checkpoints" # path to save weights
topks = eval("[20]") # @k test list
tensorboard = 1 # enable tensorboard
comment = "lgn"
# let pandas shut up
from warnings import simplefilter
simplefilter(action="ignore", category=FutureWarning)



def cprint(words : str):
    print(f"\033[0;30;43m{words}\033[0m")

# Data Loader

In [8]:
import os
from os.path import join
import sys
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from time import time

# abstracted class for every data
class BasicDataset(Dataset):
    def __init__(self):
        print("init dataset")
    
    # @property - getter / @variable.setter - setter
    @property
    def n_users(self):
        raise NotImplementedError
    
    @property
    def m_items(self):
        raise NotImplementedError
    
    @property
    def trainDataSize(self):
        raise NotImplementedError
    
    @property
    def testDict(self):
        raise NotImplementedError
    
    @property
    def allPos(self):
        raise NotImplementedError
    
    def getUserItemFeedback(self, users, items):
        raise NotImplementedError
    
    def getUserPosItems(self, users):
        raise NotImplementedError
    
    def getUserNegItems(self, users):
        """
        not necessary for large dataset
        it's stupid to return all neg items in super large dataset
        """
        raise NotImplementedError
    
    def getSparseGraph(self):
        """
        build a graph in torch.sparse.IntTensor.
        Details in NGCF's matrix form
        A = 
            |I,   R|
            |R^T, I|
        """
        raise NotImplementedError
    

In [ ]:
class gowalla(BasicDataset):
    """ 
    Dataset type for pytorch include graph information
    Gowalla Dataset
    
    """